In [1]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd

GAMES = '/kaggle/input/olympics-legacy-1896-2020/all_athlete_games.csv'
df = pd.read_csv(filepath_or_buffer=GAMES)
df['Medal rank'] = df['Medal'].map({'Gold': 1, 'Silver': 2, 'Bronze': 3})
df.head()

,Entry ID,Name,Gender,Age,Team,NOC,Year,Season,City,Sport,Event,Medal,Medal rank
0,0,A Dijiang,Male,24.0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,NaN
1,1,A Lamusi,Male,23.0,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,NaN
2,2,Gunnar Nielsen Aaby,Male,24.0,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,NaN
3,3,Edgar Lindenau Aabye,Male,34.0,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,1.0
4,26,"Cornelia ""Cor"" Aalten (-Strannood)",Female,18.0,Netherlands,NED,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN,NaN


How many athletes of each sex are in our dataset?

In [3]:
from plotly.express import histogram
histogram(data_frame=df[['Name', 'Gender', 'Season']].drop_duplicates(ignore_index=True), x='Gender', color='Season')

How has the number of medals varied over time?

In [4]:
histogram(data_frame=df[['Year', 'Medal', 'Medal rank']].dropna(subset='Medal').sort_values(by=['Medal rank'], ascending=False), x='Year', color='Medal')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286237 entries, 0 to 286236
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Entry ID    286237 non-null  int64  
 1   Name        286237 non-null  object 
 2   Gender      286237 non-null  object 
 3   Age         276763 non-null  float64
 4   Team        286237 non-null  object 
 5   NOC         286237 non-null  object 
 6   Year        286237 non-null  int64  
 7   Season      286237 non-null  object 
 8   City        286237 non-null  object 
 9   Sport       286237 non-null  object 
 10  Event       286237 non-null  object 
 11  Medal       42232 non-null   object 
 12  Medal rank  42232 non-null   float64
dtypes: float64(2), int64(2), object(9)
memory usage: 28.4+ MB


In [6]:
df.nunique()

Entry ID      271116
Name          146361
Gender             2
Age               74
Team            1196
NOC              233
Year              36
Season             2
City              42
Sport             84
Event           1071
Medal              3
Medal rank         3
dtype: int64